In [ ]:
import sys
sys.path.insert(0, "/eos/user/s/sbysiak/.local/lib/python3.7/site-packages/")
from comet_ml import Experiment

In [ ]:
# sklearn 0.22 needed for permutation importance
import sys
sys.path.insert(0, "/eos/user/s/sbysiak/.local/lib/python3.7/site-packages/")
import sklearn
sklearn.__version__

In [ ]:
import os
from time import time
from functools import partial

import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc, f1_score, roc_curve, roc_auc_score, classification_report, confusion_matrix, auc
from sklearn.inspection import permutation_importance

In [ ]:
from helper.preprocessing import add_sorting_index, add_sorted_col, add_nth_val, apply_cut
from helper.utils import convert_float64_to_float32

from helper.plotting import plot_roc, plot_score_vs_pt, plot_score_vs_col, plot_tagging_eff, plot_confusion_matrix, plot_xgb_learning_curve, plot_score_distr, plot_signal_significance, plot_eff_vs_threshold, plot_pdp
from helper.utils import signal_eff, get_optimal_threshold, convert_float64_to_float32, save_model, printmd
from helper.interpret import feature_importance_report

In [ ]:
plt.rcParams['font.size']=16
pd.options.display.max_columns = 200

# Prepare DMatrix

## Estimate available #jets and decide trainset size

In [ ]:
# def round_down(x,n=2):
#     return np.floor(x*10**n)/10**n

# # round_down(123.999, 4)

In [ ]:
tree_name_core = 'JetTree_AliAnalysisTaskJetExtractor_Jet_AKTChargedR040_tracks_pT0150_E_scheme_'
data_dir = '../ana_results/iter3/LHC16h3/'
files = [os.path.join(data_dir, d, 'AnalysisResults.root') for d in os.listdir(data_dir)]


def calc_njets(files, n_b=None, trainset_frac_b=None):
    if n_b and trainset_frac_b:
        raise ValueError('One cannot provide both parameters: `n_b` and `trainset_frac_b`')
    if not n_b and not trainset_frac_b:
        raise ValueError('Provide on out of these two parameters: `n_b` and `trainset_frac_b`')
        
    n_avail_b, n_avail_c, n_avail_udsg = 0, 0, 0
    for f in files:
        froot = uproot.open(f)
        n_avail_b += froot[tree_name_core+'bJets'].numentries
        n_avail_c += froot[tree_name_core+'cJets'].numentries
        n_avail_udsg += froot[tree_name_core+'udsgJets'].numentries
#     print(n_avail_b, n_avail_c, n_avail_udsg)

    if not n_b:
        n_b = int(n_avail_b * trainset_frac_b)
    else:
        trainset_frac_b = n_b / n_avail_b # round_down(n_b / n_avail_b, 4)
    n_c    = int(0.1 * n_b)
    n_udsg = int(0.9 * n_b)

    trainset_frac_c    = n_c / n_avail_c # round_down(n_c / n_avail_c, 4)
    trainset_frac_udsg = n_udsg / n_avail_udsg # round_down(n_udsg / n_avail_udsg, 4)
    
    d = {'b'   :[n_avail_b, n_b, trainset_frac_b], 
         'c'   :[n_avail_c, n_c, trainset_frac_c], 
         'udsg':[n_avail_udsg, n_udsg, trainset_frac_udsg]}
    df = pd.DataFrame(d)
    df.index = ['n available', 'n trainset', 'trainset fraction']
    return df
    
    
n_jets = calc_njets(files, trainset_frac_b=0.5)
n_jets

In [ ]:
n_jets.loc['trainset fraction']

## ROOT->CSV

In [ ]:
def add_features(df):

#     def IPdNSigmaAbs_cutSmallSigma(row):
#         pt = row['Jet_Track_Pt']
#         IPd_sigma = np.sqrt(row['Jet_Track_CovIPd'])
#         sigma_threshold = 0.004444561*pt**(-0.4790711) if pt < 10 else 0.0016
#         if IPd_sigma > sigma_threshold:
#             return abs(row['Jet_Track_IPd'] / IPd_sigma)
#         else:
#             return -1
    
    
    def subtract_phi(phi1, phi2):
        diff = phi1-phi2
        if abs(diff) <= np.pi: return diff
        elif diff > np.pi: return diff - 2*np.pi
        elif diff < -np.pi: return diff + 2*np.pi


    def subtract_eta(eta1, eta2):
        diff = eta1-eta2
        return diff
    
    # add custom features
#     df['Jet_Track_DeltaPhi'] = df.apply(lambda row: np.array([ subtract_phi(tr_phi, row['Jet_Phi']) for tr_phi in row['Jet_Track_Phi']]), axis=1)
#     df['Jet_Track_DeltaEta'] = df.apply(lambda row: np.array([ subtract_eta(tr_eta, row['Jet_Eta']) for tr_eta in row['Jet_Track_Eta']]), axis=1)
#     df['Jet_Track_DeltaR']   = df.apply(lambda row: np.array([ np.sqrt(tr_phi**2 + tr_eta**2)       for tr_phi, tr_eta in zip(row['Jet_Track_DeltaPhi'], row['Jet_Track_DeltaEta'])]), axis=1)
#     df['Jet_Track_PtFrac']   = df.apply(lambda row: np.array([ (tr_pt/row['Jet_Pt'])                for tr_pt in row['Jet_Track_Pt']]), axis=1)
#     df = df.drop(['Jet_Track_Phi', 'Jet_Track_Eta'])
# IPdNsigma, IPzNsigma, IP3dNsigma
# 

#     df['Jet_Track_IPdNsigmaAbs']  = df.apply(lambda row: abs(row['Jet_Track_IPd'] / np.sqrt(row['Jet_Track_CovIPd'])), axis=1)
#     df['Jet_Track_IPdNsigmaAbs']  = df.apply(lambda row: IPdNsigmaAbs_cutSmallSigma(row), axis=1)
    df['Jet_Track_IPdSigma']  = df['Jet_Track_CovIPd'].pow(0.5)
    df['Jet_Track_IPzSigma']  = df['Jet_Track_CovIPz'].pow(0.5)
    df = df.drop(['Jet_Track_CovIPd', 'Jet_Track_CovIPz'], axis=1)
    
    df['Jet_Track_IPdAbs']          = eval('abs(a)', dict(a=df['Jet_Track_IPd'])) 
    df['Jet_Track_IPzAbs']          = eval('abs(a)', dict(a=df['Jet_Track_IPz'])) 
    df['Jet_Track_IPdNsigma']       = eval('a/b', dict(a=df['Jet_Track_IPd'], b=df['Jet_Track_IPdSigma'])) 
    df['Jet_Track_IPzNsigma']       = eval('a/b', dict(a=df['Jet_Track_IPz'], b=df['Jet_Track_IPzSigma'])) 
    df['Jet_Track_IPdNsigmaAbs']    = eval('abs(a)/b', dict(a=df['Jet_Track_IPd'], b=df['Jet_Track_IPdSigma'])) 
    df['Jet_Track_IPzNsigmaAbs']    = eval('abs(a)/b', dict(a=df['Jet_Track_IPz'], b=df['Jet_Track_IPzSigma'])) 

#     def cut_val(track_pt):
#         return 0.004444561*track_pt**(-0.4790711) if track_pt < 10 else 0.0015
    
#     df['Jet_Track_CutIPdSigmaVSPt'] = df.apply(lambda row: 
#                                         np.array([int(ipd_sigma < cut_val(pt))  for ipd_sigma, pt in zip(row['Jet_Track_IPdSigma'], row['Jet_Track_Pt'])]),
#                                         axis=1
#                                       )
#     df = df.drop(['Jet_Track_IPd', ], axis=1)
#     df = df.drop(['Jet_Track_IPd', 'Jet_Track_IPz'], axis=1)
    
    df['Jet_SecVtx_LxyNsigma'] = eval('a / b', dict(a=df['Jet_SecVtx_Lxy'], b=df['Jet_SecVtx_SigmaLxy']))
    
    ### create index cols
    track_sorting_var = 'IPdNsigmaAbs'
    sv_sorting_var    = 'LxyNsigma'
    add_sorting_index(df, f'Jet_Track_{track_sorting_var}', 'desc')
    add_sorting_index(df, f'Jet_SecVtx_{sv_sorting_var}', 'desc')

    ### apply cuts a.k.a. filter index cols
#     apply_cut(df, 'Jet_Track_IPdNsigmaAbs < 50', track_sorting_var, 'desc')
#     apply_cut(df, 'Jet_Track_Pt > 0.5', track_sorting_var, 'desc')
#     apply_cut(df, 'Jet_Track_CutIPdSigmaVSPt < 0.5', track_sorting_var, 'desc')
#     apply_cut(df, 'Jet_SecVtx_Chi2 < 10' ,'LxyNsigma', 'desc')
#     apply_cut(df, 'Jet_SecVtx_Dispersion < 0.01' ,'LxyNsigma', 'desc')
#     apply_cut(df, 'Jet_SecVtx_SigmaLxy < 0.1' ,'LxyNsigma', 'desc')
    
    ### create sorted cols
    track_params = ['Jet_Track_Pt', 'Jet_Track_Phi', 'Jet_Track_Eta', 
                    'Jet_Track_DeltaPhi', 'Jet_Track_DeltaEta', 'Jet_Track_PtFrac', 'Jet_Track_DeltaR',
                    'Jet_Track_Charge', 'Jet_Track_Label', 
                    'Jet_Track_IPd', 'Jet_Track_IPz', 'Jet_Track_CovIPd', 'Jet_Track_CovIPz', 
                    'Jet_Track_ProdVtx_X', 'Jet_Track_ProdVtx_Y', 'Jet_Track_ProdVtx_Z',
                   
                    'Jet_Track_PID_ITS', 'Jet_Track_PID_TPC', 'Jet_Track_PID_TOF', 'Jet_Track_PID_TRD', 
                    'Jet_Track_PID_Reconstructed', 'Jet_Track_PID_Truth',
                    
                    'Jet_Track_IPdAbs'      , 'Jet_Track_IPzAbs',
                    'Jet_Track_IPdSigma'    , 'Jet_Track_IPzSigma',
                    'Jet_Track_IPdNsigma'   , 'Jet_Track_IPzNsigma',  
                    'Jet_Track_IPdNsigmaAbs', 'Jet_Track_IPzNsigmaAbs',
                   ]
    
    sv_params    = ['Jet_SecVtx_X', 'Jet_SecVtx_Y', 'Jet_SecVtx_Z', 
                    'Jet_SecVtx_Mass', 
                    'Jet_SecVtx_Lxy', 'Jet_SecVtx_SigmaLxy', 'Jet_SecVtx_Chi2', 'Jet_SecVtx_Dispersion', 'Jet_SecVtx_LxyNsigma',
                   ]
    
    track_params = [par for par in track_params if par in df.columns]
    sv_params    = [par for par in  sv_params   if par in df.columns]

    for param in track_params:
        add_sorted_col(df, param ,   track_sorting_var, 'desc')

    for param in sv_params:
        add_sorted_col(df, param ,   sv_sorting_var, 'desc')

    
    ### extract n-th value from sorted cols
#     new_training_cols = []
    n_tracks, n_sv = 10,3
    for param in track_params:
        for i in range(n_tracks):
            add_nth_val(df, col_name=f'{param}__sortby__{track_sorting_var}__desc', n=i, fillna=None)
#             new_training_cols.append(df.columns[-1])

    for param in sv_params:
        for i in range(n_sv):
            add_nth_val(df, col_name=f'{param}__sortby__{sv_sorting_var}__desc', n=i, fillna=None)
#             new_training_cols.append(df.columns[-1])

    ### drop temporary columns, i.e. those containing arrays, like 'Index__*' as well as initial columns used for extraction, like 'Jet_Track_Pt'
#     columns_to_keep = df.select_dtypes(exclude=['object']).columns
    columns_to_keep = [col for col,val in zip(df.columns, df.iloc[0]) if not hasattr(val, '__iter__') or isinstance(val, str)]
    return df[columns_to_keep]


In [ ]:
branches_to_read = ['Jet_Pt', 
                 'Jet_Phi', 'Jet_Eta', 
                 'Jet_Area', 'Jet_NumTracks',
#                  'Jet_Track_Pt', 
#                  'Jet_Track_Phi', 'Jet_Track_Eta', 
                 'Jet_Track_IPd','Jet_Track_IPz', 'Jet_Track_CovIPd', 'Jet_Track_CovIPz',
#             'Jet_Track_PID_ITS', 'Jet_Track_PID_TPC', 'Jet_Track_PID_TOF', 'Jet_Track_PID_TRD', 'Jet_Track_PID_Reconstructed', 'Jet_Track_PID_Truth',
            'Jet_SecVtx_Mass', 'Jet_SecVtx_Lxy', 'Jet_SecVtx_SigmaLxy', 'Jet_SecVtx_Chi2', 'Jet_SecVtx_Dispersion',

#             'Jet_Shape_Mass_NoCorr', 'Jet_Shape_Mass_DerivCorr_1', 'Jet_Shape_Mass_DerivCorr_2',
#             'Jet_Shape_pTD_DerivCorr_1', 'Jet_Shape_pTD_DerivCorr_2', 'Jet_Shape_LeSub_NoCorr', 'Jet_Shape_LeSub_DerivCorr',
#             'Jet_Shape_Angularity', 'Jet_Shape_Angularity_DerivCorr_1', 'Jet_Shape_Angularity_DerivCorr_2',
#             'Jet_Shape_Circularity_DerivCorr_1', 'Jet_Shape_Circularity_DerivCorr_2', 'Jet_Shape_Sigma2_DerivCorr_1', 'Jet_Shape_Sigma2_DerivCorr_2',
#             'Jet_Shape_NumTracks_DerivCorr', 'Jet_Shape_MomentumDispersion', 'Jet_Shape_TrackPtMean', 'Jet_Shape_TrackPtMedian',
                ]

froot = uproot.open(os.path.join('../ana_results/iter3/LHC16h3/ptbin1/AnalysisResults.root'))
df = froot[tree_name_core+'bJets'].pandas.df(flatten=False, branches=branches_to_read)#.query('Jet_Pt > 10 and Jet_Pt < 100')
print('tree reading done')
df_after = add_features(df)
print('features extracted')

In [ ]:
n_jets = calc_njets(files, n_b=2000000)
n_jets

In [ ]:
# df_b = froot[tree_name_core+'bJets'].pandas.df(flatten=False, branches=branches_to_read, entrystart=istart, entrystop=istop).query(query_str)
#             if len(df) < 2: continue
#             df = convert_float64_to_float32(df)
#             df = add_features(df)

In [ ]:
data_tmp_key = ''.join(np.random.choice([l for l in 'abcdefghijklmnopqrstuvwxyz1234567890'], 6))
n_jets.to_csv(f'n_jets_{data_tmp_key}.csv')

In [ ]:
fractions = n_jets.loc['trainset fraction']
n_chunk = 10000
fname = f'train_{data_temp_key}.csv'
flavour_map = {'b':1, 'c':0, 'udsg':0}
offset = 0

open(fname, 'w').close() # clear file

is_first = True
for f in files:
    froot = uproot.open(f)
    tic = time()
    for flavour in ['b', 'c', 'udsg']:
        n_avail = froot[tree_name_core+f'{flavour}Jets'].numentries
        n_read = int(n_avail * fractions[flavour])
        print(f'Reading {n_read:6d} {flavour:>4s}-jets from {f}')
        for i in range(int(n_read/n_chunk)+1):
            istart = i*n_chunk + offset
            istop = min((i+1)*n_chunk, n_read) + offset
#             print(f'\t\t\t {istart:7d} -- {istop:7d}')
            df = froot[tree_name_core+f'{flavour}Jets'].pandas.df(flatten=False, branches=branches_to_read, entrystart=istart, entrystop=istop)
            if len(df) < 2: continue
            df = convert_float64_to_float32(df)
            df = add_features(df)
            df.insert(loc=0, column='label', value=flavour_map[flavour])
            df.to_csv(fname, mode='a', index=False, header=is_first)
            is_first=False
    print(f'\t\t{f} processed in {time()-tic:.1f} sec')

In [ ]:
fractions = n_jets.loc['trainset fraction']
n_chunk = 10000
fname = f'test_{data_temp_key}.csv'
flavour_map = {'b':1, 'c':0, 'udsg':0}

open(fname, 'w').close() # clear file

is_first = True
for f in files:
    tic = time()
    froot = uproot.open(f)
    for flavour in ['b', 'c', 'udsg']:
        n_avail = froot[tree_name_core+f'{flavour}Jets'].numentries
        offset = int(n_avail * fractions[flavour])
        n_read = int(n_avail * fractions[flavour]/5)
        print(f'Reading {n_read:6d} {flavour:>4s}-jets from {f}')
        for i in range(int(n_read/n_chunk)+1):
            istart = i*n_chunk + offset
            istop = min((i+1)*n_chunk, n_read) + offset
#             print(f'\t\t\t {istart:7d} -- {istop:7d}')
            df = froot[tree_name_core+f'{flavour}Jets'].pandas.df(flatten=False, branches=branches_to_read, entrystart=istart, entrystop=istop)
            if len(df) < 2: continue
            df = convert_float64_to_float32(df)
            df = add_features(df)
            df.insert(loc=0, column='label', value=flavour_map[flavour])
            df.to_csv(fname, mode='a', index=False, header=is_first)
            is_first=False
    print(f'\t\t{f} processed in {time()-tic:.1f} sec')

## CSV->DMatrix

In [ ]:
fname = 'train_2M-b.csv'

In [ ]:
[c for c in pd.read_csv(fname, nrows=1).columns if '0' in c or 'Track' not in c and 'SecVtx' not in c]

In [ ]:
# [c for c in df.columns if 'Track_0_' in c]

In [ ]:
!rm tmp/dtrain.cache tmp/dtest.cache

In [ ]:
dtrain = xgb.DMatrix(f'{fname}?format=csv&label_column=0#tmp/dtrain.cache')
# l = dtrain.num_row()
# idx = [i for i in range(0,l,10)]
# dtrain = dtrain.slice(idx)
dtest = xgb.DMatrix(f'{fname.replace("train","test")}?format=csv&label_column=0#tmp/dtest.cache')

In [ ]:
dval = dtest

In [ ]:
# dtrain = dtrain_eval

### Train-validation split (if there is one csv file available)

In [ ]:
# dtrain_eval = dtrain
val_frac = 0.1
n = dtrain_val.num_row()
val_idx = np.random.choice(n, size=int(val_frac*n), replace=False)

In [ ]:
# HIGHLY INEFFECTIVE
train_idx = [i for i in np.arange(n) if i not in val_idx]

In [ ]:
val_idx

In [ ]:
len(val_idx), len(train_idx), dtrain_val.num_row()

In [ ]:
dval, dtrain = dtrain_val.slice(val_idx), dtrain_val.slice(train_idx)

# Create experiment and log data info

In [ ]:
# !pip install --user --no-cache-dir --upgrade comet_ml

In [ ]:
try: 
    exp.end()
except:
    pass
exp = Experiment(
                 auto_output_logging='simple',
                 log_env_gpu=False, log_env_cpu=False,
                 project_name="test", workspace="phd")

In [ ]:
columns = pd.read_csv(fname, nrows=1).columns
print(columns.to_list())
print('\n\n')
for c in [col for col in columns if ('Jet_Track_' not in col and 'Jet_SecVtx_' not in col) or '0' in col]: print(c)
n_tracks = max([int(col.split('_')[2]) for col in columns if 'Jet_Track_' in col])+1
n_sv     = max([int(col.split('_')[2]) for col in columns if 'Jet_SecVtx' in col])+1

In [ ]:
y = dtrain.get_label()
print(f'train set: b: {sum(y==1)}, rest: {sum(y==0)}')
yv = dval.get_label()
print(f'valid set: b: {sum(yv==1)}, rest: {sum(yv==0)}')

In [ ]:
exp.add_tags(['b-vs-rest'])
# if nrows_c == nrows_udsg: exp.add_tag('N_c = N_udsg')
# else: exp.add_tag('N_c =/= N_udsg')

y = dtrain.get_label()
exp.log_other('n_jets_b', sum(y==1))
exp.log_other('n_jets_rest', sum(y==0))
# exp.log_other('n_jets_c', n_c_jets)
# exp.log_other('n_jets_udsg', n_udsg_jets)

exp.log_other('n_columns', dtrain.num_col())
exp.log_other('n_rows', dtrain.num_row())
exp.log_other('n_jets', dtrain.num_row())
exp.log_parameter('n_tracks', n_tracks)
exp.log_parameter('n_sv', n_sv)

# n_jets_str = f'there is:\n\t{n_b_jets} b jets\n\t{n_c_jets} c jets\n\t{n_udsg_jets} udsg jets'
dataset_info = f'ncols={dtrain.num_col()}, nrows={dtrain.num_row()}, ntr={n_tracks}, nsv={n_sv}'
print(dataset_info)
exp.log_dataset_info(dataset_info)
exp.log_dataset_hash(dtrain)

In [ ]:
# exp.add_tag('full-info')
# custom_descr = '2M-b, perf, overfit3X: lr=0.5, depth=15, n_est=10
custom_descrtom_descr = 'DUMMY'
exp.log_other('descr', f'{custom_descr} ; {dataset_info}')

# Train model

## Check performance of XGBClassifier trained on pd.DataFrame

In [ ]:
# df_b = pd.read_csv('datasets/iter2/bjets_10-150GeV_Tr-sortbyIPdNsigmaAbs-noCuts_SV-sortbyLxyNsigma-noCuts.csv', nrows=50000)
# df_udsg = pd.read_csv('datasets/iter2/udsgjets_10-150GeV_Tr-sortbyIPdNsigmaAbs-noCuts_SV-sortbyLxyNsigma-noCuts.csv', nrows=50000)
# df_b['label'] = 1
# df_udsg['label'] = 0
# df = pd.concat([df_b, df_udsg], axis=0)
# df.head()

In [ ]:
# # df = pd.read_csv(fname)
# print(fname)
# df = pd.read_csv(fname)
# y = df['label']
# X = df.drop('label', axis=1)
# if 'ptbin' in X.columns: X = X.drop('ptbin', axis=1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=122)

In [ ]:
# params = dict(n_estimators=2, learning_rate=0.2, 
#               max_depth=5, tree_method='exact', 
#               gamma=10, reg_lambda=0,
#               subsample=0.8, colsample_bytree=0.8, colsample_bynode=0.8,
#               scale_pos_weight=(sum(y==0)/sum(y==1)), random_state=123,
#              )
    
# # exp.add_tag('XGB')
# # exp.log_parameters(params, prefix='manual')  # backward compatibility
# # exp.log_parameters(params, prefix='man')
# clf = XGBClassifier(**params)
# clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], 
# #         eval_metric=partial(xgb_callback, make_plots=True), 
#         verbose=10)

In [ ]:
# y_train_proba = clf.predict_proba(X_train)[:,1]
# y_test_proba = clf.predict_proba(X_test)[:,1]

# print('roc_auc_test', roc_auc_score(y_test, y_test_proba))
# print('roc_auc_train', roc_auc_score(y_train, y_train_proba))

## XGBoost.train()

In [ ]:
training_iter = 0

y_train = dtrain.get_label()
y_test = dtest.get_label()



def xgb_callback(y_pred, dtrain, mistag_rates=[0.1, 0.01, 0.001], make_plots=False):
    global training_iter
    y_true = dtrain.get_label()
    metrics = []
    for mistag_rate in mistag_rates:
        metrics.append((f'bEff@mistag_{mistag_rate:.0e}', signal_eff(y_true, y_pred, mistag_rate)))
#     metrics.append(('ROC_AUC', roc_auc_score(y_true, y_pred)))
    if any([' ' in met_name or ':' in met_name for met_name, _ in metrics]):
        raise ValueError('Metric names cannot contain space nor colon(:)')

    if not make_plots: 
        return metrics
    is_testset = False
    if len(y_true) == len(y_test):
        is_testset = all(y_true == y_test)
    if (not (training_iter % 30)) or training_iter in [0,1,3]:
        if not is_testset:
            ax = plot_tagging_eff(y_true, y_pred, label='train', color='r' if is_testset else 'b')
        else:
            ax = plot_tagging_eff(y_true, y_pred, label='test', color='r' if is_testset else 'b', ax=plt.gca())
            ax.set_ylim(1e-4, 2)
            exp.log_figure(f'plot_iter{training_iter:04}')        
    if is_testset:
        training_iter += 1        
    return metrics


params = dict(objective='binary:logistic',
              n_estimators=100,
              learning_rate=0.1, 
              max_depth=15, tree_method='approx', 
              min_split_loss=10, reg_lambda=0, # aka gamma, L2
              subsample=0.8, colsample_bytree=1, colsample_bynode=1,
              scale_pos_weight=sum(y==0)/sum(y==1), 
#               random_state=123,
              eval_metric=['auc', 'logloss', ]
             )
    
exp.add_tag('XGB')
exp.log_parameters(params, prefix='manual')  # backward compatibility
exp.log_parameters(params, prefix='man')

watchlist = [(dtrain, 'train'),(dval, 'eval'),]
eval_res = {}

tic = time()
bst = xgb.train(params, dtrain, params['n_estimators'], 
                watchlist, evals_result=eval_res, 
                feval=partial(xgb_callback, make_plots=True))

exp.log_other('training time', int(time()-tic))

In [ ]:
bst.save_model('tmp.model')

## Eval training

In [ ]:
# eval_res = clf.evals_result()
for metric in eval_res[list(eval_res.keys())[0]].keys():
    print(metric)
    ax = plot_xgb_learning_curve(eval_res, metric)
    exp.log_figure(f'{metric}_vs_ntrees')

## Save model

In [ ]:
# save_model(clf, X.columns, scaler, exp, 'xgb_model')

# TODO:

- **saving & loading**
    - requires: booster.save_model(), list of input files, fractions, add_features() func, (list of columns), ...
    - should work with:   
        1) dynamically reading from ROOT  
        2) from ready csv -- such file has to have all required fields associated with it, e.g. zipped
- learning curves  -- DONE
- reading dataframes from CSV -- DONE
- callbacks -- plots & logging of -- DONE
- pT-weighted test set (at least assign weights and pass when logging metrics: auc and signal eff)  
  check how many splittings come from low pt jets, in given Erad bins
- investigate differences in pT between train and test set

# Report performance -- _bulk_  (based on just scores)

## Inference

In [ ]:
y_train = dtrain.get_label()
y_train_proba = bst.predict(dtrain)

y_test = dtest.get_label()
y_test_proba = bst.predict(dtest)

In [ ]:
opt_thresh = get_optimal_threshold(y_train, y_train_proba, 0.04)
y_train_pred_opt = (y_train_proba > opt_thresh).astype('int')
y_test_pred_opt  = (y_test_proba  > opt_thresh).astype('int')

In [ ]:
mistag_rates = [0.1, 0.01, 0.001]
for mistag_rate in mistag_rates:
    eff = signal_eff(y_test, y_test_proba, mistag_rate)
    exp.log_metric(f'tagEff@mistag_{mistag_rate:.0e}', eff)

## Scores distribution

In [ ]:
# # raise NotImplementedError('TODO')
fig,axes = plt.subplots(nrows=2, figsize=(12,7), gridspec_kw={'height_ratios': [2,1]})
plot_score_distr(y_train, y_train_proba, linestyle=':', ax=axes[0])
plot_score_distr(y_test , y_test_proba , linestyle='-', ax=axes[0], lw=2)
plot_signal_significance(y_train, y_train_proba, 0.02,    linestyle=':', color='cyan'   ,  label='b frac. = 2%', ax=axes[1])
plot_signal_significance(y_train, y_train_proba, 0.04,    linestyle=':', color='lime'   ,  label='b frac. = 4%', ax=axes[1])
plot_signal_significance(y_train, y_train_proba, 0.08,    linestyle=':', color='magenta',  label='b frac. = 8%', ax=axes[1])
# plot_signal_significance(y_test , y_test_proba , 0.01,    linestyle='-', color='lime'   , lw=2, label='b frac. = 1%', ax=axes[1])
# plot_signal_significance(y_test , y_test_proba , 0.04,    linestyle='-', color='magenta', lw=2, label='b frac. = 4%', ax=axes[1])
axes[0].vlines(opt_thresh, *axes[0].get_ylim(), color='lime', lw=2, linestyle=':')
exp.log_figure('score_and_significance_vs_threshold')

xmax = max(max(y_train_proba), max(y_test_proba))
axes[0].set_xlim(xmax-0.2, xmax+0.01)
axes[1].set_xlim(xmax-0.2, xmax+0.01)
axes[1].set_ylim(0.95,1)
exp.log_figure('score_and_significance_vs_threshold_zoom')

## eff vs threshold

In [ ]:
ax = plot_eff_vs_threshold(y_test, y_test_proba)
exp.log_figure('eff_vs_threshold')
ax.set_xlim(0.6,1)
exp.log_figure('eff_vs_threshold_zoom')
ax.set_yscale('log')
exp.log_figure('eff_vs_threshold_logy')

## ROC - log AUC scores and plot vs pT

In [ ]:
ax = plot_roc(y_train, y_train_proba, label='train', color='b');
ax = plot_roc(y_test, y_test_proba, label='test' , color='r', ax=ax);
exp.log_figure('roc_curve')

In [ ]:
exp.log_metric('roc_auc_test', roc_auc_score(y_test, y_test_proba))
exp.log_metric('roc_auc_train', roc_auc_score(y_train, y_train_proba))

## mistagging rate VS tagging efficiency

In [ ]:
ax = plot_tagging_eff(y_test, y_test_proba, label='$b$ vs $c+udsg$ test', color='r')
plot_tagging_eff(y_train, y_train_proba, label='$b$ vs $c+udsg$ train', color='b', ax=ax)
exp.log_figure('tagging_eff')

In [ ]:
mistag_rates = [0.1, 0.01, 0.001]
for mistag_rate in mistag_rates:
    eff = signal_eff(y_test, y_test_proba, mistag_rate)
    exp.log_metric(f'tagEff@mistag_{mistag_rate:.0e}', eff)

In [ ]:
# exp.end()

## Confusion matrices

In [ ]:
printmd('__TRAIN__')
fig, axes = plt.subplots(ncols=2, figsize=(10,5))
fig.tight_layout()
fig.subplots_adjust(wspace=0.5)
plot_confusion_matrix(y_train, y_train_pred_opt, ['c+udsg', 'b'], title='train, unnormalized', normalize=False, ax=axes[0])
plot_confusion_matrix(y_train, y_train_pred_opt, ['c+udsg', 'b'], title='train, normalized'  , normalize=True , ax=axes[1])
exp.log_figure('confusion_matrix_train')

In [ ]:
printmd('__TEST__')
fig, axes = plt.subplots(ncols=2, figsize=(10,5))
fig.tight_layout()
fig.subplots_adjust(wspace=0.5)
plot_confusion_matrix(y_test, y_test_pred_opt, ['c+udsg', 'b'], title='test, unnormalized', normalize=False, ax=axes[0])
plot_confusion_matrix(y_test, y_test_pred_opt, ['c+udsg', 'b'], title='test, normalized'  , normalize=True , ax=axes[1])
exp.log_figure('confusion_matrix_test')

# Report performance -- _bulk_ on $p_T$-weigthed scores

# Report performance -- _differential_  (based on full dataframes)

In [ ]:
del y_train, y_test, y_train_proba, y_test_proba

In [ ]:
clf = XGBClassifier()
clf.load_model('tmp.model')
clf.classes_ = np.array([0,1])

# https://github.com/dmlc/xgboost/issues/2073
from sklearn.preprocessing import LabelEncoder
clf._le = LabelEncoder().fit([0,1])

In [ ]:
def wc(fname):
    from subprocess import check_output
    return int(check_output(f'wc -l {fname}', shell=True).decode('utf-8').split(' ')[0])

n_read = 200000

n_rows_total   = wc(fname)
read_every_nth = int(n_rows_total / n_read)
df_train = pd.read_csv(fname, nrows=n_read, skiprows=lambda x: x%read_every_nth)
y_train = df_train['label']
X_train = df_train.drop('label', axis=1)
if 'ptbin' in X_train.columns: X_train = X_train.drop('ptbin', axis=1)

n_rows_total   = wc(fname.replace('train', 'test'))
read_every_nth = int(n_rows_total / n_read)
df_test = pd.read_csv(fname.replace('train', 'test'), nrows=n_read, skiprows=lambda x: x%read_every_nth)
y_test = df_test['label']
X_test = df_test.drop('label', axis=1)
if 'ptbin' in X_test.columns: X_test = X_test.drop('ptbin', axis=1)

In [ ]:
y_train_proba = clf.predict_proba(X_train)[:,1]
y_test_proba = clf.predict_proba(X_test)[:,1]

y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [ ]:
len(y_train_pred), len(y_test_pred)

## Performance vs _feature_

In [ ]:
# from importlib import reload
# import helper
# reload(helper)
# reload(helper.plotting)
# reload(helper.plotting.performance_plots)
# plot_score_vs_col = helper.plotting.plot_score_vs_col
# plot_pdp = helper.plotting.plot_pdp

In [ ]:
def aver_pred(y_true, y_score):
    return np.average(y_score)


for feature,bins,bins_distplot in [
                     ('Jet_Pt', (5,10,15,20,30,40,50,60,80,100), 100), 
                     ('Jet_Phi', 18*3, 10), 
                     ('Jet_Eta', 20, 100), 
                     ('Jet_NumTracks', np.arange(0,30,2), np.arange(0,30,1)),
                      ]:
    if feature not in X_train.columns: continue
    for score in [(roc_auc_score, 'ROC AUC'), 
                  (partial(signal_eff, mistag_rate_thresh=1e-2), 'signal eff for mistag=1e-2'),
                  (partial(signal_eff, mistag_rate_thresh=3e-2), 'signal eff for mistag=3e-2'),
#                   (aver_pred, 'aver. score')
                    ]:
        ax=plot_score_vs_col(y_train, y_train_proba, 
#                       vals=scaler.inverse_transform(X_train)[:, df.columns.get_loc(feature) ], 
                      X_train[feature],
                      bins=bins, bins_distplot=bins_distplot,
                      score=score, color='b', label='train',
                      show_distplot=True,                          
                      show_errorbars=True,
                     )
        plot_score_vs_col(y_test, y_test_proba, 
#                           vals=scaler.inverse_transform(X_test)[:, df.columns.get_loc(feature) ],
                          X_test[feature],
                          bins=bins, bins_distplot=bins_distplot,
                          score=score, color='r', marker='^', label='test', 
                          xlabel=feature,
                          show_distplot=True,
                          show_errorbars=True,
                          ax=ax
                         )
        exp.log_figure(f"{score[1].replace(' ', '').replace('=','-').replace('.', '-')}_vs_{feature.replace('Jet', '')}");



# Interpretability

## Feature importance

**XGB's weight**   
  = how many times feature was used

In [ ]:
imp_dict = clf.get_booster().get_score(importance_type='weight')  # default only till xgboost-0.90
imp = imp_dict.values()
names = imp_dict.keys()
# names = X.columns[[int(k[1:]) for k in imp_dict.keys()]]
feature_importance_report(imp, names, importance_type='XGB\'s weight')

**XGB's total gain**

In [ ]:
imp_dict = clf.get_booster().get_score(importance_type='total_gain')
imp = imp_dict.values()
names = imp_dict.keys()
# names = X.columns[[int(k[1:]) for k in imp_dict.keys()]]
feature_importance_report(imp, names, importance_type='XGB\'s total_gain')

**Permutation importance**  
remember to use scaled input data  
it also quite time-consuming

In [ ]:
imp = permutation_importance(clf,X_train[::20],y_train[::20])['importances_mean']
feature_importance_report(imp, X_train.columns, importance_type='permutation imp.')
perm_imp = imp
perm_imp_feats = X_train.columns

## Partial dependence plots
for 5 features with highest permutation importance

In [ ]:
# class dummy_scaler():
#     def __init__(self):
#         pass
#     def transform(self,x):
#         return x
#     def inverse_transform(self,x):
#         return x

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# from sklearn.inspection import partial_dependence
# from helper.plotting._utils import _add_distplot


# def plot_pdp(clf, X, feature, scaler=None, column_names=None, query=None, 
#               xlabel=None, show_deciles=True, show_distplot=False, y=None, 
#               pardep_kws={}, plt_kws={}, distplot_kws={}, ax=None):
#     """ plots partial dependence plot against `feature` for samples satifying `query`

#     Parameters
#     ----------
#     clf : compatible with sklearn.inspect.parital_dependence
#         model
#     X : pd.DataFrame or 2D array of numbers
#         data to calculated partial dependece
#         both training and hold-on set (or combined) is reasonable for this purpose
#     feature : str
#         name of the feature to compute pdp w.r.t
#     scaler : sklearn-compatible scaler e.g. StandardScaler
#         scaler used to scale training data
#     column_names : iterable of strings
#         names of the columns in the dataset,
#         if None (default) then X has to be dataframe with correct columns
#         other args as `feature` or `query` relies on it
#     query : str
#         selection criteria, only samples passing it will be used to compute pdp
#         has to be valid input for pd.DataFrame.query()
#     xlabel : str or None
#         xlabel, if None (default) `feature` will be used
#     show_deciles : bool
#         if small vertical lines (seaborn's rugs) corresponding to deciles of selected `feature` values should be shown,
#         selected i.e. passing `query`
#     show_distplot : bool
#         if distribution of `feature` should be plotted below pdp
#         if `y` passed, than it's grouped by y=0/1
#     y : array of numbers
#         samples labels, used to split distplot, 
#         used only if show_distplot is True
#     pardep_kws : dict
#         passed to sklearn.inspect.parital_dependence
#     plt_kws : dict
#         passed to plt.plot
#     distplot_kws : dict
#         passed to sns.distplot, 
#         has some defaults - see code
#     ax : matplotlib.axes._subplots.AxesSubplot object or None
#         axes to plot on
#         default=None, meaning creating axes inside function

#     Returns
#     -------
#     ax
#     """
#     if not ax:
#         _,ax = plt.subplots(figsize=(7,5))
#     if column_names is None:
#         column_names = X.columns


#     X_orig = scaler.inverse_transform(X) if scaler else X 
#     df = pd.DataFrame(X_orig)
#     df.columns = column_names
#     if query: df = df.query(query)

#     df_xgb = pd.DataFrame(scaler.transform(df)) if scaler else df
#     if feat not in clf.get_booster().feature_names:
#         df_xgb.columns = [f'f{i}' for i in range(df_xgb.shape[1])]
#         feat_idx = list(column_names).index(feature)
#         feat_name_xgb = f'f{feat_idx}'
#     else: 
#         df_xgb.columns = df.columns
#         feat_idx = list(column_names).index(feature)
#         feat_name_xgb = feature
        
#     part_dep, feat_vals = partial_dependence(clf, df_xgb[df_xgb[feat_name_xgb].notna()], features=[feat_name_xgb], **pardep_kws)
#     part_dep, feat_vals = np.array(part_dep[0]), np.array(feat_vals[0])
#     if scaler: feat_vals_orig = feat_vals * np.sqrt(scaler.var_[feat_idx]) + scaler.mean_[feat_idx]
#     else: feat_vals_orig = feat_vals

#     ax.plot(feat_vals_orig, part_dep, lw=3, **plt_kws)
#     ax.set_xlim(left=min(ax.get_xlim()[0], min(feat_vals_orig)), right=max(ax.get_xlim()[1], max(feat_vals_orig)))

#     vals = df[feature]
#     if show_deciles:
#         xlim = ax.get_xlim()
#         deciles = np.nanpercentile(vals, np.arange(0, 101, 10))
#         sns.rugplot(deciles, ax=ax)
#         ax.set_xlim(xlim)
#     if show_distplot:
#         distplot_default_kws = dict(bins=np.linspace(*ax.get_xlim(),100), distplot_y_frac=0.8)
#         distplot_kws = {**distplot_default_kws, **distplot_kws}  # passed `distplot_kws` overwrites defaults
#         _add_distplot(ax, vals, y=y, **distplot_kws)        

#     ax.set_xlabel(xlabel if xlabel else feature)
#     ax.set_ylabel('partial dependence')
#     return ax


In [ ]:
most_imp_idx = np.argsort(perm_imp)[:-10:-1]
features = X_train.columns[most_imp_idx]
for feat in features:
    ax = plot_pdp(clf, X_train[::10], feat, 
             scaler=None, 
             column_names = X_train.columns,
             query='',
             show_deciles=True,
             show_distplot=True,
             y=y_train[::10],
             pardep_kws=dict(percentiles=(0.1,0.9)),
            )
    exp.log_figure(f"pdp_{feat}")



### Check pt distr train/test

In [ ]:
X_train['Jet_Pt'].plot(figsize=(20,3))

In [ ]:
for pt in [X_train['Jet_Pt'], X_test['Jet_Pt']]:
    print(pt.mean(), pt.median(), np.percentile(pt,75))

In [ ]:
for pt in [X_train['Jet_Pt'][::2], X_train['Jet_Pt'][1::2]]:
    print(pt.mean(), pt.median(), np.percentile(pt,25))

## Model explainers

In [ ]:
code = '\n#\n'.join(In)
exp.set_code(code=code)
In.clear()
exp.end()